## Chronos & LoRA: Learning fine details efficiently :)
Before diving into fine-tuning, let's get your environment ready with the following steps:
1. **Create a Python virtual environment:**
    <pre>
    `python -m venv [name of the environment]`
    </pre>
2. Open a terminal inside the LoRA folder (located within the repository), and install Chronos in editable mode with training dependencies:
    <pre>
    `pip install --editable ".[training]"`
    </pre>
3. **Install PEFT (Parameter-Efficient Fine-Tuning)** to enable LoRA support: 
    <pre>
    `pip install peft`
    </pre>
4. **Install the CUDA-enabled version of PyTorch** to leverage GPU acceleration (if available): 
    - If you are on **Windows 11** with an NVIDIA GPU: 
    <pre>
        `pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128`
    </pre>
    - On other platforms, please visit the [official PyTorch install guide](https://pytorch.org/get-started/locally/) to find the right command for your system
5. **Start fine-tuning:** Navigate to the scripts folder and run:
`python training/train.py --config ./training/configs/chronos-t5-small-lora.yaml`.
    <pre>
   This command uses the configuration specified in: `./training/configs/chronos-t5-small-lora.yaml`
    </pre>

In [ ]:
import datasets
from gluonts.dataset.arrow import ArrowWriter
from pathlib import Path

d:\TODO\PoliTo\YEARS\Second year\Deep Natural Language Processing\Project\DeepNLP_Forecasting\LoRA\LoRAenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
def to_chronos_format(dataset: datasets.arrow_dataset.Dataset, t = None):
    import numpy as np
    return dataset.map(lambda ts: 
                {'start': str(ts['timestamp'][0]), 
                 'target': ts['target'][:t]
                 }, 
                 remove_columns=['id', 'timestamp'])

In [15]:
monash_australian_electricity_ds = datasets.load_dataset("autogluon/chronos_datasets", "monash_australian_electricity", split="train")
monash_australian_electricity_ds.set_format("numpy")  # sequences returned as numpy arrays
monash_australian_electricity_ds, type(monash_australian_electricity_ds)

(Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 5
 }),
 datasets.arrow_dataset.Dataset)

In [37]:
monash_australian_electricity_ds_for_chronos = to_chronos_format(monash_australian_electricity_ds, 128)
monash_australian_electricity_ds_for_chronos

Map: 100%|██████████| 5/5 [00:00<00:00, 127.94 examples/s]


Dataset({
    features: ['target', 'start'],
    num_rows: 5
})

In [38]:
ArrowWriter(compression="lz4").write_to_file(
        monash_australian_electricity_ds_for_chronos,
        path=Path.cwd()/ "example.arrow",
    )

In [ ]:
!python training/train.py --config ./training/configs/chronos-t5-small-lora.yaml

^C


: 